In [ ]:
# qiskit imports and magic
%matplotlib inline
from qiskit import IBMQ, execute, transpile
from qiskit import QuantumCircuit
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_histogram

In [22]:
#if you're on a mac, you'll need some special magic for py5
%gui osx

In [23]:
# py5 imports
import time
import py5
import py5_tools

In [49]:
# some important variables
line_points = 1000000
qubits = 7 # this is determined by the backend we chose, mostly
iterations = 5
lines = 20

In [50]:
# Quantum backend setup

# I'm using a fake noisy backend (Aer simulator + noise model from terra's testing code). 
# You can use real hardware by importing 
# The provider code instead, but I'd recommend prototyping like this
from qiskit.test.mock import FakeGuadalupe
guadalupe = FakeGuadalupe()
noisy_backend = QasmSimulator.from_backend(guadalupe)

# let's also register a "noise free" simulator for comparison later
perfect_backend = QasmSimulator()

In [51]:
# build a circuit builder
def circuit_builder(builder_qubits, builder_iterations):
    circ = QuantumCircuit(builder_qubits, builder_qubits)
    for i in range(builder_iterations):
        for q in range(qubits-1):
            circ.cx(q, q+1)
    circ.measure_all(add_bits=False)
    return circ

In [52]:
circ = circuit_builder(qubits, iterations)
print(circ)

                                                                            ░ »
q_0: ──■─────────■─────────■─────────■─────────■────────────────────────────░─»
     ┌─┴─┐     ┌─┴─┐     ┌─┴─┐     ┌─┴─┐     ┌─┴─┐                          ░ »
q_1: ┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■───────────────────────░─»
     └───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐                     ░ »
q_2: ─────┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──────────────────░─»
          └───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐                ░ »
q_3: ──────────┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■─────────────░─»
               └───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐           ░ »
q_4: ───────────────┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■────────░─»
                    └───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐      ░ »
q_5: ────────────────────┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■───░─»
                         └───┘┌─┴─┐└───┘

In [53]:
#run a circuit
perf_result = perfect_backend.run(transpile(circ, perfect_backend), shots=line_points, memory=True).result()
perf_counts = perf_result.get_counts()
print(perf_counts)

noisy_result = noisy_backend.run(transpile(circ, noisy_backend), shots=line_points, memory=True).result()
noisy_counts = noisy_result.get_counts()
print(noisy_counts)

# get them as a shot list instead
perf_list = perf_result.get_memory()
noisy_list = noisy_result.get_memory()

{'0000000': 1000000}
{'1101011': 731, '0010011': 1097, '1100011': 943, '0001011': 963, '0001101': 1135, '1001001': 1177, '0111101': 1159, '0011101': 927, '0011011': 835, '1011010': 2499, '1000011': 2104, '1110111': 2523, '0001010': 2063, '0110110': 2329, '0010010': 1740, '1111000': 14950, '0101001': 1580, '0000101': 1356, '1011111': 3236, '1110101': 2603, '1000010': 2613, '1111101': 1394, '0011001': 1876, '1110011': 845, '0101111': 1635, '0101110': 2433, '0111011': 1109, '1001101': 1311, '1101101': 884, '0001001': 1724, '1010011': 1098, '1010001': 2821, '0010101': 1388, '0111001': 1084, '1101100': 1912, '1010101': 5004, '0011010': 1658, '1101110': 5129, '0100110': 2387, '1111011': 818, '1101010': 1542, '0000110': 4775, '1001010': 1840, '0001100': 2172, '1000000': 17503, '0001110': 2943, '0101011': 1125, '1111010': 1450, '1001111': 3562, '0011110': 6234, '0010110': 4349, '0011000': 13268, '1011101': 1371, '0010111': 1634, '0100010': 5020, '1010000': 9688, '1111100': 4993, '0010100': 238

In [54]:
def convert_list_to_int(list):
    return([int(x, 2) for x in list])

In [56]:
def setup():
    py5.size(1000, 1000)
    py5.no_loop()

def draw():
    py5.background(255)
    black_alpha = py5.color(0, 128)
    margin = 50
    line_height = (py5.height - (margin*2))/lines
    noisy_list_int = convert_list_to_int(noisy_list)
    
    py5.load_pixels()
    for i in range(len(py5.pixels)): #iterate over every pixel starting top left and scanning like a CRT
            py5.pixels[i] = py5.color(0, noisy_list_int[i])
    
    py5.update_pixels()
    

py5.run_sketch(block=False)
print('the sketch is running!')

Sketch is already running. To run a new sketch, exit the running sketch first.
the sketch is running!


In [58]:
portal = py5_tools.sketch_portal(quality=75, scale=1.0)
portal

Output(layout=Layout(max_height='200px', overflow='auto'))

In [ ]:
#some handy py5 snips

# portal = py5_tools.sketch_portal(quality=75, scale=1.0)
# portal

# py5.hot_reload_draw(draw2)

# py5.frame_count

py5.exit_sketch()